In [71]:
import numpy as np
import pandas as pd
import os.path
import subprocess
import math
import datetime
import matplotlib
from copy import copy, deepcopy
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.dates as mdates
from datetime import datetime, timedelta

In [72]:
#settings
start_date=datetime(2017,12,6)
num_days=34
start_date_scen=start_date+timedelta(days=num_days)
num_days_scen=14

In [73]:

dtdz=-0.006       # temperature gradient (deg C km-1)
hstat=1709         # elevation of weather station
topo=[0.78,1768,44,0.0172]        # area, elevation, max depth, volume
elev_lake=topo[1]     # elevation of lake

# ---- scenario - INPUT / ADJUSTMENT / CALIBRATION
scenprec_corrfact=1.0              # Factor to correct precipitation in second week of scenario - normally strongly overestimated...

ice_thickness_scen=25              # (cm) Ice thickness (measured) at start of scenario period  (default: -9999)
#ice_thickness_scen=-np.inf

snow_thickness_scen=3             # (cm w.==!) snow thickness (measured) at start of scenario period  (default: -9999)
#snow_thickness_scen=-np.inf

snow_dens_scen=400                # (kg m-3) snow density (measured/estimated)  at start of scenario period  (default: -9999)
#snow_dens_scen=-np.inf

slushice_thickness_scen=-np.inf               # (cm) thickness of refrozen ice ("Sandwich") at start of scenario period  (default: -9999)
#slushice_thickness_scen=-np.inf

# ------------------


format_csv='y'      # raw climate file in csv format

# prepare climate data file
prepare='n'

twater_ini=[4,4,20]       # [Tmin,Tmax,break(m)]

int_slush=[datetime(2018,1,3),datetime(2018,1,4),datetime(2018,1,9)]       # dates to insert slush

vol_slush=[0.003,0.017,0.01,-np.inf]            # volumes of inserted slush in m w.e. (same length as int_slush)

dtdz=-0.006       # temperature gradient (deg C km-1)
hstat=1709         # elevation of weather station

alpha_ice=0.3         # albedo of ice
alpha_water=0.12      # albedo of water
alpha_snow0=0.92      # albedo of fresh snow
alpha_snow1=0.5       # albedo of very old snow

turbulent_fact=3.5  # 5 # factor to increase transfer of water heat to base of ice (to be calibrated)

corrprec=1      # Korrektur des Niederschlags

lakedepth=25     # depth of lake m

first_freezeparam=1.0        # [-] Parameter to accelerate or decelerate the first freezing of the lake

# temperature of river entering the lake and lake geometry
topo=[0.78,1768,44,0.0172]        # area, elevation, max depth, volume
flow=1     #5.8                    # [m3/s] river charactersitic inflow
flow_tcorr=-5    # [deg C] correct temperature of stream flow for elevation / snow melt relative to air temperature
flow_tmin=0      # [deg C] minimum temperature of stream

wwin=[1.2,1,0.8]     # weighting of lake water layering in different depths

elev_lake=topo[1]     # elevation of lake


# number of iterations  for heat conduction (per hour)
iterations_si=600     # snow / ice
iterations_w=50     # water

# number of iterations for snow surface temperature
dt_snoit=120    # s

freq_plots=6     # (hours) frequency of output plots
language_plots='e'      # language of plots (d: deutsch, e: english)


# -----------------------------
# parameters / constants

kice=2.33       # conductivity ice [J s-1 K-1 m-1]
kwater=0.56     # conductivity water
kair=0.001

cice=1890000     # heat capacity of ice
cair=1297
cwater=4217700    # [J m-3 K-1]

emissivity_atm=0.95
emissivity_ice=0.966
emissivity_water=0.957
emissivity_snow=0.97

attenuation_water=1       # m-1
attenuation_ice0=1.0         # m-1
attenuation_snow=10     # m-1   ??? 10
at_rho=250
fact_at=0.1                # increase in ice attenuation per day (ageing)

rho=917   # ice density
rho_water=1000
rho_snow0=70    # density of fresh snow
rho_max=400

compression='y'
rho_compression=450    # kg m-3


L=334000*rho    # [J m-3] latent heat of fusion
Lhe=2260000    # [J kg-1] latent heat of evaporation
Lhs=2594000    # [J kg-1] latent heat of sublimation

bulk_exchange_s=0.3*pow(10,-3)      # bulk exchange coefficients for sensible heat flux (to be calibrated)
bulk_exchange_l=0.8*pow(10,-3)      # bulk exchange coefficients for latent heat flux (to be calibrated)

bulk_exchange_s=1.5*pow(10,-3)      # bulk exchange coefficients for sensible heat flux (to be calibrated)
bulk_exchange_l=1.5*pow(10,-3)      # bulk exchange coefficients for latent heat flux (to be calibrated)


# -----------------------
# settings

dz=0.005      # nodespacing (ice)
dzs=0.0025    # snow
dzw=0.2      # water

nodes=[lakedepth/dzw,300,300]     # water, ice, snow

dt=3600     # [s] timestep

s_b=5.67*pow(10,-8)    # stefan boltzmann

t_wmin=4     # minimum temperature of water (density)

meltlast=0

In [74]:
df=pd.read_csv('../data/interim/scenario.csv')
df['When']=df['Date'] + ' ' + df['Time']
df['When'] = pd.to_datetime(df['When'],format='%Y-%m-%d %H:%M:%S')
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time'])
df.tail()

,Unnamed: 0,Rad,Tair,RH,Wind,Pressure,Prec,Cl,Date,Time,huma,rho_air,When
1147,1147,0.0,-7.4,78.8,1.3,82610.0,0.000156,0.40335,2018-01-22,2018-05-21 19:00:00,0.00158,1.09758,2018-01-22 19:00:00
1148,1148,0.0,-7.4,75.1,1.4,82660.0,0.000156,0.40335,2018-01-22,2018-05-21 20:00:00,0.00168,1.09241,2018-01-22 20:00:00
1149,1149,0.0,-9.8,75.6,0.6,82630.0,0.000156,0.40335,2018-01-22,2018-05-21 21:00:00,0.00178,1.08954,2018-01-22 21:00:00
1150,1150,0.0,-9.8,80.2,0.8,82590.0,0.000156,0.40335,2018-01-22,2018-05-21 22:00:00,0.00195,1.08736,2018-01-22 22:00:00
1151,1151,0.0,-9.8,94.7,0.7,82560.0,0.000156,0.40335,2018-01-22,2018-05-21 23:00:00,0.00230,1.08697,2018-01-22 23:00:00


In [75]:
#variables
ice=np.full(nodes[1],-np.inf)
snow=np.full((3,nodes[2]),-np.inf)
nodes=list(map(int, nodes))
fr=0
sn=0
im=0
y=0
c=0
i_slush=0
Qbmelt=0
Qswi_i=0.0
ci_icest=0
water=np.zeros(shape=nodes[0])
water+=twater_ini[0]

for i in range(nodes[0]-1,-1,-1):
    d=i*dzw
    if d< twater_ini[2]:
        water[i]=water[i]+((((twater_ini[2]/dzw)-i)*(twater_ini[1]-twater_ini[0]))/(twater_ini[2]/dzw))

slush=np.full((3,nodes[2]),0.0)
icethick=np.full(df.shape[0],0.0)
load=np.full((2,df.shape[0]),0.0)
mtemp=np.full((3,df.shape[0]),0.0)
stype=np.full(df.shape[0],0.0)
snowthick=deepcopy(stype)
icet=deepcopy(stype)
snow_albedo=deepcopy(stype)
slush_thick=deepcopy(icethick)
i_age=np.full(df.shape[0],-np.inf)

dfo=pd.DataFrame({'Type' : []})
dfo['Temp']=-np.inf
dfo['Date']=df['When'].dt.date
dfo['Time']=df['When'].dt.time
dfo['Icethick']=0
dfo['Snowthick']=0

In [76]:
with PdfPages('../data/processed/figures.pdf') as pdf:
    for index in range(0,df.shape[0]):
        ii=np.where(snow[0,]!=-np.inf)
        ci=ii[0].size
        jj=np.where(ice!=-np.inf)
        cj=jj[0].size
        hh=np.where(water!=-np.inf)
        ch=hh[0].size
        if ci>0:
            tt=snow[0,ci-1]
            print(df.loc[index,'When'],'Snow')
            print(snow[0,0:ci],np.sum(slush[0,]))
            dfo.loc[index,'Type']='Snow'

        if ci==0 and cj>1:
            tt=ice[0]
            print(df.loc[index,'When'],'Ice')
            print(ice[0:5],np.sum(slush[0,]))
            dfo.loc[index,'Type']='Ice'
        if cj==0:
            tt=water[0]
            print(df.loc[index,'When'].day,'Water')
            print(water[0:5])
            dfo.loc[index,'Type']='Water'
        dfo.loc[index,'Temp']=tt

        if np.isnan(tt) or tt==-np.inf:
            print(snow[0,])
            exit()

        ew=6.1094*np.exp((17.625 * tt)/(tt+243.04))*100
        hum0=0.622*ew/abs(df.loc[index,'Pressure']-ew)
    # *************************************************
    # *************************************************
    # SNOW
        tes=deepcopy(snow[0,])
        tls=deepcopy(tes)

        # accumulation
        t_threshold=1.25
        print("Prec", df.loc[index,'Prec'])
        ptt=df.loc[index,'Prec']
        if df.loc[index,'Tair']>t_threshold+1:
            df.loc[index,'Prec']=0
        if df.loc[index,'Tair']<=t_threshold+1 and df.loc[index,'Tair']>t_threshold-1:
            df.loc[index,'Prec']=df.loc[index,'Prec']*(1-(df.loc[index,'Tair']-(t_threshold-1))/2)

        df.loc[index,'p_liq']=ptt-df.loc[index,'Prec'] #Liquid Prec

        if ice[0]==-np.inf:
            df.loc[index,'Prec']=0
        sn=sn+df.loc[index,'Prec'] #Solid Prec
        print("Snow,Water", sn, df.loc[index,'p_liq'])

        if sn>=dzs:
            if snow[0,0]==-np.inf:
                snow[0,0]=ice[0]
            sn=sn-dzs
            l=np.where(snow[0,]==-np.inf)
            l=l[0]
            snow[0,l[0]]=min([df.loc[index,'Tair'],0])
            snow[1,l[0]]=rho_snow0/1000

            #artificial compression of snow
            if compression=='y':
                snow[1,l[0]]=rho_compression/1000

            snow[2,l[0]]=1/24

        #update snow density and albedo
        ts_snow=21.9
        l=np.where(snow[0,]!=-np.inf)
        l=l[0]
        if l.size!=0:
            for i in range(0,l.size):
                if snow[1,i]<rho_max/1000 or snow[1,i]==-np.inf:
                    snow[1,i]=(rho_max-(rho_max-rho_snow0)*math.exp(-0.03*pow((0.01*rho_max),0.5)*snow[2,i]))/1000
                    snow[1,0]=0.2

                if snow[2,i]!=-np.inf:
                    alpha_snow=alpha_snow1+(alpha_snow0-alpha_snow1)*math.exp((-1)*snow[2,i]/ts_snow)
                if snow[2,i]!=-np.inf:
                    snow[2,i]=snow[2,i]+1/24

        else:
            alpha_snow=0.85

        # arrays for capacity and conductivity depending on density for snow
        csnow=np.full(nodes[2],0.0)
        ksnow=deepcopy(csnow)
        for i in range(0,nodes[2]):
            if snow[0,i] != -np.inf:
                csnow[i]=(1-snow[1,i])*cair+snow[1,i]*cice
            if snow[0,i] != -np.inf:
                ksnow[i]=(2.93*pow(snow[1,i],2))+0.01   # according to Mellor (1977)

        # update capacity and conductivity if slush is present
            if slush[0,i]!= 0:
                c=slush[1,i]*cwater+(1-slush[1,i])*cice
                k=slush[1,i]*kwater+(1-slush[1,i])*kice                
                ff=(slush[0,i]/slush[2,i])*(1-snow[1,i])
                csnow[i]=(1-snow[1,i]-ff)*cair+snow[1,i]*cice+ff*c
                ksnow[i]=(1-snow[1,i]-ff)*kair+snow[1,i]*kice+ff*k

        # -------  energy balance snow
        Qswi=(1-alpha_snow)*df.loc[index,'Rad']
        Qswo=(-1)*alpha_snow*df.loc[index,'Rad']   # outgoing short wave

        Qlwi=(0.68+0.0036*pow(df.loc[index,'huma'],0.5))*(1+0.18*pow(df.loc[index,'Cl'],2))*emissivity_atm*s_b*pow((df.loc[index,'Tair']+273.15),4)
        Qe=df.loc[index,'rho_air']*Lhs*bulk_exchange_l*(df.loc[index,'huma']-hum0)*df.loc[index,'Wind']
        if df.loc[index,'Tair']> 1 :
            Qp=df.loc[index,'p_liq']*(df.loc[index,'Tair']-1)*cwater/dt
        else:
            Qp=0
        gamma=math.exp(-attenuation_snow*dzs/(at_rho/1000.))
        # iterate for snow surface temperature
        if snow[0,0]== -np.inf :
            dt_snoit_tt=1
        else:
            dt_snoit_tt=dt_snoit

        snit=dt/dt_snoit_tt
        l=np.where(snow[0,] != -np.inf)
        l=l[0]
        ci=l.size
        snit=int(snit)
        for it in range(0,snit):
            if snow[0,0]==-np.inf :
                tsno=0
            else:
                if it==0:
                    tsno=snow[0,l[ci-1]]

                else:
                    tsno=tt_ts
                tt_ts=tsno

            Qlwo=(-1)*emissivity_snow*s_b*pow((tsno+273.15),4)
            Qc=df.loc[index,'rho_air']*cair*bulk_exchange_s*(df.loc[index,'Tair']-tsno)*df.loc[index,'Wind']
            Q0snow=(1-gamma)*Qswi+Qlwi+Qlwo+Qc+Qe+Qp

        # calculate temperature of uppermost layer with energy balance
            if ci > 1 :
                tt_ts=tt_ts+Q0snow*dt_snoit_tt/(csnow[l[ci-1]]*(dzs/snow[1,l[ci-1]]))

        # done iteration for snow surface temperature

        l=np.where(snow[0,] != -np.inf)
        l=l[0]
        ci=l.size
        if ci > 1 :
            # attribute surface temperature
            tls[l[ci-1]]=tt_ts
            tls[0]=ice[0]

            for i in range(ci-2,0,-1):
                tls[l[ci-2]]=tls[l[ci-1]]+(math.exp(-attenuation_snow*(i+1)*dzs/(at_rho/1000))-math.exp(-attenuation_snow*i*dzs/(at_rho/1000)))*Qswi*dt/(csnow[l[ci-1]]*(dzs/snow[1,l[ci-1]]))

            Qswi_s=Qswi*math.exp(-attenuation_snow*(ci-1)*dzs/(at_rho/1000))

            # ----- snow evaporation
            if Qe < 0 :
                sn=sn+Qe*dt/(Lhs*1000)

            # ----- snow melt
            if Q0snow > 0 and 0<=tls[l[ci-1]] :
                Qmelt=Q0snow+(tls[l[ci-1]]*csnow[l[ci-1]]*(dzs/snow[1,l[ci-1]])/dt)
                melt=Qmelt/L*dt
                if slush[1,l[ci-1]]!=0:
                    m=melt*(dzs/(dzs+slush[0,l[ci-1]]*(slush[1,l[ci-1]])))
                else:
                    m=melt
                sn=sn-m
                tls[l[ci-1]]=0
            else:
                melt=meltlast
            
            # reduce snow
            if sn <= -dzs :
                sn=sn+dzs
                tls[l[ci-1]]=-np.inf
            print("III",melt)
            l=np.where(tls != -np.inf)
            l=l[0]
            ci=l.size
            # add liquid precipitation to melt water
            melt=melt+df.loc[index,'p_liq']
            print("II",melt)
            # ----- refreezing of melt water
            if melt > 0:
                for j in range(ci-2,0,-1) :
                    re=(-1)*snow[0,j]*csnow[j]*(dzs/snow[1,j])/L
                    if melt > re :
                        tls[j]=0
                    elif re > 0 :
                        tls[j]=(1-melt/re)*snow[0,j]
                    if melt > re :
                        melt=melt-re
                    elif re > 0:
                        melt=0
            print("I",melt)
            # ----- slush formation (melt water in snow pore space)
            if melt > 0 :
                for j in range(1,ci):
                    sp=dzs/snow[1,j]*(1-snow[1,j])
                    a=sp-slush[0,j]
                    slush[2,j]=sp
                    if melt < a and melt != 0 :
                        slush[0,j]=slush[0,j]+melt
                        melt=0
                        slush[1,j]=1
                        print("GG",np.sum(slush[0,]))
                    if melt > a :
                        melt=melt-a
                        slush[0,j]=sp
                        print("GGG",np.sum(slush[0,]))
                    if melt < 0 :
                        melt=0
                    if slush[0,j] < 0 :
                        slush[0,j]=0
                        print("GGGG",np.sum(slush[0,]))
            
            jj=np.where(snow[0,] == -np.inf)
            jj=jj[0]
            slush[0,jj]=0
            if jj.size>0:
                if jj[0]<= 1 :
                    slush[0,0:2]=dzs
            
            # ----- manual slush injection (melt water in snow pore space) - injection at 12.00
            if vol_slush[y] != -np.inf :
                if df.loc[index,'Date'] in int_slush:
                    ij=vol_slush[y]
                    y=y+1
                    for j in range(1,ci) :
                        sp=dzs/snow[1,j]*(1-snow[1,j])
                        a=sp-slush[0,j]
                        if ij > 0 :
                            if ij < a :
                                slush[0,j]=slush[0,j]+ij
                                ij=0
                                slush[1,j]=1
                                slush[2,j]=sp
                            else:
                                slush[0,j]=slush[0,j]+a
                                ij=ij-a
                                slush[1,j]=1
                                slush[2,j]=sp

            # ----- refreezing of slush
            for j in range(1,ci) :
                if slush[0,j] != 0 and slush[1,j] > 0 and tls[j] < 0 :
                    re=(-1)*tls[j]*csnow[j]*(dzs/snow[1,j])/L
                    if (slush[0,j]*slush[1,j]) > re :
                        tls[j]=0
                        slush[1,j]=slush[1,j]-(re/(slush[0,j]*slush[1,j]))*slush[1,j]
                    else:
                        if re > 0 :
                            tls[j]=(1+(slush[0,j]*slush[1,j])/re)*tls[j]
                            slush[1,j]=0
            l=np.where(tls != -np.inf)
            l=l[0]
            ci=l.size
            tes=deepcopy(tls)

            # -----------------------
            # heat conduction
            
            tll=deepcopy(tls)
            tel=deepcopy(tes)
            for it in range(0,iterations_si) :
                for j in range(0,ci-1) :
                    if j == 0 :
                        tel[j]=tll[j]-(dt/iterations_si*ksnow[j]/(csnow[j])*(tll[j]-tll[j+1]))/(pow((dzs/snow[1,j]),2))/2
                    else:
                        tel[j]=tll[j]+((dt/iterations_si*ksnow[j]/(csnow[j])*(tll[j-1]-tll[j])/(pow((dzs/snow[1,j]),2)))-(dt/iterations_si*ksnow[j]/(csnow[j])*(tll[j]-tll[j+1])/pow((dzs/snow[1,j]),2)))/2
                tll=deepcopy(tel)

            tes=deepcopy(tel)
            ice[0]=tes[0]   # setting ice surface temperature to value of lowermost snow layer!
            snow[0,]=deepcopy(tes)
            snow[0,0]=tls[0]

            if l.size > 0 :
                snow[0,l[ci-1]]=tls[l[ci-1]]
            meltlast=melt

        # correct and reset
        if snow[0,1] == -np.inf :
            snow.T[0,]=-np.inf
        l=np.where(np.logical_and(snow[0,]!=-np.inf,snow[0,]<-40))
        l=l[0]

        if l.size > 0 :
            snow[0,l]=-40

        l=np.where(np.logical_and(snow[0,]!=-np.inf,snow[0,]>0))
        l=l[0]
        if l.size > 0 :
            snow[0,l]=0

        # ----  forcing layers to measured thicknesses at begin of scenario period
        if df.loc[index,'When']==start_date_scen:
            if snow_thickness_scen != -np.inf:
                l=np.where(snow[0,]!=-np.inf)
                l=l[0]
                if l.size > 0 :
                    snow_thickness_scen_ind=int(snow_thickness_scen/dzs/100)
                    if l.size < snow_thickness_scen_ind:
                        for h in range(l.size,snow_thickness_scen_ind):
                            snow.T[h,]=snow.T[l[l.size-1],]
                    else:
                        for h in range(snow_thickness_scen_ind, l.size):
                            snow.T[h,]=-np.inf
            if snow_dens_scen !=-np.inf and snow_thickness_scen !=-np.inf:
                l=np.where(snow[0,]!=-np.inf)
                l=l[0]
                if l.size > 0 :
                    snow_thickness_scen_ind=int(snow_thickness_scen/dzs/100)
                    for h in range(0,snow_thickness_scen_ind):
                        snow[1,h]=snow_dens_scen/1000

            if slushice_thickness_scen != -np.inf:
                l=np.where(snow[0,]!=-np.inf)
                l=l[0]
                if l.size > 0 :
                    slushice_thickness_scen_ind=int(slushice_thickness_scen/dzs/100)
                    if l.size<slushice_thickness_scen_ind:
                        for h in range(l.size,slushice_thickness_scen_ind):
                            slush.T[h,]=slush.T[l[l.size-1],]
                    else:
                        for h in range(slushice_thickness_scen_ind,l.size):
                            slush.T[h,]=0
        
        # *************************************************
        # ICE
        # age of ice sheet
        ii=np.where(ice != -np.inf)
        ii=ii[0]
        ci=ii.size
        jj=np.where(ice == -np.inf)
        jj=jj[0]
        i_age[jj]=-np.inf
        for i in range(0,ci):
            if i_age[i] == -np.inf:
                i_age[i]=1/24
            else:
                i_age[i]=i_age[i]+1/24

        if ci > 0 :
            attenuation_ice=attenuation_ice0+np.mean(i_age[ii])*fact_at
        else:
            attenuation_ice=attenuation_ice0

        # energy balance ice
        Qswi=(1-alpha_ice)*df.loc[index,'Rad']
        Qswo=(-1)*alpha_ice*df.loc[index,'Rad']
        Qlwi=(0.68+0.0036*pow(df.loc[index,'huma'],0.5))*(1+0.18*pow(df.loc[index,'Cl'],2))*emissivity_atm*s_b*pow((df.loc[index,'Tair']+273.15),4)
        Qe=df.loc[index,'rho_air']*Lhs*bulk_exchange_l*(df.loc[index,'huma']-hum0)*df.loc[index,'Wind']                                     #huma==hum0???
        Qp=0
        gamma=math.exp(-attenuation_ice*dz)
        
        # iterate for ice surface temperature
        if snow[0,1] == -np.inf :
            dt_iceit=dt_snoit
        else:
            dt_iceit=dt
        snit=(dt/dt_iceit)

        for it in range(0,int(snit)) :
            if ice[0] == -np.inf :
                tice=0
            else:
                tice=ice[0]
            if it > 1 :
                tice=tt_ts
            else:
                tt_ts=tice

            Qlwo=(-1)*emissivity_ice*s_b*pow((tice+273.15),4)     # long-wave outgoing
            Qc=df.loc[index,'rho_air']*cair*bulk_exchange_s*(df.loc[index,'Tair']-tice)*df.loc[index,'Wind']   # sensible heat flux
            Q0ice=(1-gamma)*(Qswi)+Qlwi+Qlwo+Qc+Qe+Qp          # energy balance of top layer
            tt_ts=tt_ts+Q0ice*dt_iceit/(cice*dz)

        l=np.where(ice != -np.inf)
        l=l[0]
        ci=l.size
        if ci > 0 :
            ii=np.where(ice != -np.inf)
            ii=ii[0]
            ci=ii.size
        
            te=deepcopy(ice)
            tl=deepcopy(te)
            # calculate temperature of uppermost layer with energy balance
            if snow[0,1] == -np.inf :
                tl[0]=tt_ts# apply surface temperature from iteration

                # adapting ice temperature based on direct radiation absorbed within ice sheet
                
                for i in range(1,ci-1):
                    tl[i]=tl[i]+(math.exp(-attenuation_ice*(i-1)*dz)-math.exp(-attenuation_ice*i*dz))*Qswi*dt/(cice*dz)
                Qswi_i=Qswi*math.exp(-attenuation_ice*(ci-2)*dz)    # radiation below ice sheet
                
            # below snow coverage - temperature change calculated with heat fluxes from above and below
            if snow[0,1] != -np.inf :
                for i in range(0,ci-1):
                    tl[i]=tl[i]+(math.exp(-attenuation_ice*(i-1)*dz)-math.exp(-attenuation_ice*i*dz))*Qswi_s*dt/(cice*dz)
                Qswi_i=Qswi_s*math.exp(-attenuation_ice*(ci-2)*dz)   # radiation below ice sheet

                snow[0,0]=tl[0]     # equalizing ice surface and bottom snow temperature
            
            # ----- ice evaporation
            if Qe < 0 and snow[0,1] == -np.inf :
                if 0<= i_slush :
                    im=im+Qe*dt/(Lhs*917)
                else:
                    i_slush=i_slush+Qe*dt/(Lhw*1000)
                if i_slush < 0 :
                    i_slush=0

            # ----- ice melt
            if Q0ice > 0 and tl[0] >= 0 :
                Qmelt=Q0ice+(tl[0]*cice*dz/dt)
                if Qmelt > 0 :
                    imelt=Qmelt/L*dt
                else:
                    imelt=0
                im=im-imelt
                tl[0]=0
                i_slush=i_slush+imelt
            else:
                melt=0

            # reduce ice cover
            if im <= -dz :
                im=im+dz
                for i in range(0,ci-1):
                    ice[i]=ice[i+1]
                ice[ci-1]=-np.inf
                l=np.where(ice != -np.inf)
                l=l[0]
                ci=l.size
                tl=deepcopy(ice)
            # ---- freezing of slush at ice surface
            if slush[0,1] > 0 and tl[0] < 0 :
                re=(-1)*tl[0]*cice*dz/L

                if slush[0,1]*slush[1,1] > re :
                    tl[0]=0
                    slush[1,1]=slush[1,1]*(re/(slush[0,1]*slush[1,1]))
                else:
                    if re > 0 :
                        tl[0]=tl[0]+tl[0]*((slush[0,1]*slush[1,1])/re)
                    slush[1,1]=0

            # ---- freezing liquid water on bare ice surface or within ice sheet!
            if i_slush > 0 :
                re=0
                for j in range(0,ci) :
                    if tl[j] < 0 :
                        re=(-1)*tl[j]*cice*dz/L
                    if i_slush > re :
                        tl[j]=0
                        i_slush=i_slush-re
                    else:
                        if re > 0 :
                            tl[j]=tl[j]+tl[j]*(i_slush/re)
                        i_slush=0
            ii=np.where(ice != -np.inf)
            ii=ii[0]
            ci=ii.size
            # ------- heat conduction
            tll=deepcopy(tl)
            tel=deepcopy(te)
            for it in range(0,iterations_si) :
                for j in range(1,ci-1) :
                    tel[j]=tll[j]+(((dt/iterations_si*kice/(cice)*(tll[j-1]-tll[j+1])*pow(dz,-2)))-(((dt/iterations_si*kice/(cice)*(tll[j]-tll[j+1])*pow(dz,-2)))))/2
                tll=deepcopy(tel)
            te=deepcopy(tel)
            jj=np.where(tl == -np.inf)
            jj=jj[0]
            cj=jj.size
            if cj > 0 :
                te[jj]=-np.inf
                
            # --- check for positive ice temperatures
            jj=np.where(te>0)
            jj=jj[0]
            cj=jj.size
            
            hh=np.where(np.logical_and(te<0,te!=-np.inf))
            hh=hh[0]
            ch=hh.size
            
            if cj > 0 and ch == 0 :
                fr=fr-np.sum(te[jj])*cice*dz/L
                te[jj]=0
            
            if cj > 0 and ch > 0 :
                p=np.sum(te[jj])/ch
                kk=np.where(np.logical_and(te < -p, te != -np.inf) )
                kk=kk[0]
                ck=kk.size
                if ck > 0 :
                    te[kk]=te[kk]+np.sum(te[jj])/ck
                else:
                    fr=fr-np.sum(te[jj])*cice*dz/L
                    te[jj]=0

            # ---  freezing / thawing at base!
            if ci > 1 :
                ice_flux=(-1)*min(te[ii])*cice*dz/dt
            else:
                ice_flux=0.
            if ice_flux < 0 :
                ice_flux=0
            if ci > 1 :
                fr=fr+(ice_flux+(turbulent_fact*kwater*(water[0]-water[1])/(dzw))-Qbmelt)*dt/L
            else:
                tl[0]=-np.inf
            if fr > dz :
                fr=fr-dz
                te[ci]=0
            if fr < (-1)*dz :
                fr=fr+dz
                te[ci-1]=-np.inf
                print("Melted")
                if ci > 1 :
                    te[ci-2]=0
            ice=deepcopy(te)
            ice[0]=tl[0]   # ice loop
            
        l=np.where(ice != -np.inf)
        l=l[0]
        ci=l.size
            

        if ice[0] == -np.inf :
            ice[1]=-np.inf
        if snow[0,0] == -np.inf :    #Correction
            snow[0,1]=-np.inf
        
        
        # Break up of ice cover if it is completely temperate and reasonably thin
        l=np.where(ice != -np.inf)
        l=l[0]
        ci=l.size
        if ci > 0 and ci < 4 :
            if min(ice[l]) == 0 :
                ice[l]=-np.inf
        
        
        
        #correct and reset
        ii=np.where(np.logical_and(ice != -np.inf, ice<-30))
        ii=ii[0]
        ci=ii.size
        if ci>0:
            ice[ii]=-30
        
        #forcing ice thickness at beginning of scenario period
        if df.loc[index,'When']==start_date_scen:
            if ice_thickness_scen != -np.inf:
                ice_thickness_scen_ind=int(ice_thickness_scen/dz/100)
                hh=np.where(np.logical_and(ice != -np.inf, ice<-30))
                hh=hh[0]
                ch=hh.size
                if ch < ice_thickness_scen_ind:
                    for h in range(ch,ice_thickness_scen_ind):
                        ice[h]=0
                else:
                    for h in range(ice_thickness_scen_ind,ch):
                        ice[h]=-np.inf
        
        # *************************************************
        # *************************************************
        # WATER

        # iterate for snow surface temperature
        ts_water=water[0]
        # energy balance water
        Qswi=(1-alpha_water)*df.loc[index,'Rad']
        Qswo=(-1)*alpha_water*df.loc[index,'Rad']   
        Qlwo=(-1)*emissivity_water*s_b*pow((ts_water+273.15),4)
        Qlwi=(0.68+0.0036*pow(df.loc[index,'huma'],0.5))*(1+0.18*pow(df.loc[index,'Cl'],2))*emissivity_atm*s_b*(pow(df.loc[index,'Tair']+273.15,4))
        Qc=df.loc[index,'rho_air']*cair*bulk_exchange_s*(df.loc[index,'Tair']-ts_water)*df.loc[index,'Wind']
        Qe=df.loc[index,'rho_air']*Lhe*bulk_exchange_l*(df.loc[index,'huma']-hum0)*df.loc[index,'Wind']
        Qp=0
        gamma=math.exp(-attenuation_water*dz)
        Q0water=Qswi+Qlwi+Qlwo+Qc+Qe+Qp
        l=np.where(water != -np.inf)
        l=l[0]
        ci=l.size
        tew=deepcopy(water)
        tlw=deepcopy(tew)
        
        # mixing of water during summer - 	include layering of water!
        if ice[0] == -np.inf :
            ww=np.full(ci,1.0)
            ww[0:11]=wwin[0]
            ww[10:21]=wwin[1]
            ww[ci-10:ci]=wwin[2]
            ww=ww/np.mean(ww)
            cj=ci
            if water[0] < t_wmin :
                ww=np.full(ci,0.0)
                ww[0]=1
                cj=1
            if Q0water < 0 :
                jj=np.where(water > t_wmin)
                jj=jj[0]
                cj=jj.size
                ww=np.full(ci,0.0)
                if cj > 0 :
                    ww[jj]=1
                else:
                    ww[0]=1
                if cj == 0 :
                    cj=1

            for j in range(0,cj):
                tlw[j]=tlw[j]+Q0water*dt/(cwater*cj*dzw)*ww[j]

            # !!! heat input by rivers and rain !!!
            tr=(df.loc[index,'Tair']+flow_tcorr)
            if tr < flow_tmin :
                tr=flow_tmin
            di_temp=tr-np.mean(water)
            hrain=(df.loc[index,'Prec']*corrprec)*1000000.*topo[0]*di_temp  # cwater gekuerzt!
            tlw=tlw+hrain/(topo[3]*pow(10,9))
            ran_riv=168
            tt=min([ran_riv,index])
            triv=np.mean(df.loc[index-tt:index+1,'Tair'])+flow_tcorr
            if triv < flow_tmin :
                triv=flow_tmin
                di_temp=triv-np.mean(water)
            hriv=flow*dt*di_temp  # cwater gekuerzt!
            tlw=tlw+hriv/(topo[3]*pow(10,9))
        
        #If Ice Covered
        if ice[0] != -np.inf :
            tlw[0]=0
            Qbmelt=Qswi_i*(1-gamma)
            for i in range(1,ci):
                tlw[i]=tlw[i]+(math.exp(-attenuation_water*(i-1)*dz)-math.exp(-attenuation_water*i*dz))*Qswi_i*dt/(cwater*dzw)

        tll=deepcopy(tlw)
        tel=deepcopy(tew)
        
        for it in range(0,iterations_w):
            for j in range(1,ci-1) :
                tel[j]=tll[j]+((dt/iterations_w*kwater/(cwater)*(tll[j-1]-tll[j])/pow(dzw,2))-(dt/iterations_w*kwater/(cwater)*(tll[j]-tll[j+1])/pow(dzw,2)))/2
            tll=deepcopy(tel)
        tew=deepcopy(tel)
        tew[0]=tlw[0]

        # try to eliminate values < t_wmin
        for j in range(0,ci) :
            if tew[j] < t_wmin and max(tew) > t_wmin :
                tp=t_wmin-tew[j]
                tew[j]=t_wmin

                for h in range(j+1,ci) :
                    if tew[h] > t_wmin :
                        if tp > tew[h]-t_wmin :
                            tp=tp-(tew[h]-t_wmin)
                            tew[h]=t_wmin
                        else:
                            tew[h]=tew[h]-tp
                            tp=0
        water=deepcopy(tew)
        water[0]=tlw[0]

        # first ice formation
        tfreeze=(-1)*dz*L/(dzw*cwater)*first_freezeparam
        if np.mean(water[0:2]) < tfreeze and ice[0] == -np.inf :
            ice[0:3]=0
            water[0]=0

        # *******************************
        # compile results
        snow_albedo[index]=alpha_water
        ii=np.where(ice != -np.inf)
        ii=ii[0]
        ci=ii.size
        if ci > 1 :
            icethick[index]=(ci-1)*dz

        if ci > 1 :
            mtemp[1,index]=np.mean(ice[ii[0]:ii[ci-2]])
        else:
            mtemp[1,index]=-np.inf
        if ci > 0 :
            stype[index]=1
        if ci > 0 :
            snow_albedo[index]=alpha_ice

        mtemp[0,index]=np.mean(water)

        l=np.where(snow[0,] != -np.inf)
        l=l[0]
        ci=l.size
        for i in range(1,ci):
            snowthick[index]=snowthick[index]+dzs/snow[1,i]
        if ci >= 1:
            snow_albedo[index]=alpha_snow
        if ci > 1 :
            mtemp[2,index]=np.mean(snow[0,l[0]:l[ci-1]])

        else:
            mtemp[2,index]=-np.inf
        if ci >= 1 :
            stype[index]=2

        hh=np.where(slush[1,] == 0)
        hh=hh[0]
        ch=hh.size
        if ch > 0 :
            slush_thick[index]=np.sum(slush[0,hh])*1.25
        else:
            slush_thick[index]=0

        load[0,index]=(ci-1)*dzs*1000+np.sum(slush[0,])*1000   # kg m-2
        a=7000000*pow(icethick[index],2)
        b=350000*pow(icethick[index],2)
        if load[0,index]*9.81 > b :
            load[1,index]=1

        # ********************************************
        # ********************************************
        # PLOT
        # one plot every day
        if np.mod(index,freq_plots)==0:
            plt.xlim(-40, 16)
            plt.ylim(-0.6, 0.5)
            plt.ylabel('Depth (m)')
            plt.xlabel('Temperature (deg C)')
            plt.axvline(x=0,color='k',linewidth=0.5).set_dashes([5])
            plt.axhline(y=0,color='k',linewidth=0.5)
            plt.title(str(df.loc[index,'Date'].date())+' Time '+str(df.loc[index,'When'].strftime("%H-%M")))

            if ice[0] != -np.inf:
                co=0
            else:
                co=4

            ii=np.where(ice != -np.inf)
            ii=ii[0]
            ci=ii.size
            if ci > 0:
                yi=[float(i)*dz*(-1) for i in range(0,ci)]
                xi=ice[0:ci]
                plt.plot(xi,yi,color='r',linewidth=1)
                plt.axhline(y=min(yi),color='g',linewidth=0.5)

            if ci > 0:
                offs=ii[ci-1]*dz*(-1)
            else:
                offs=0
            yw=[float(i)*dz*(-1)+offs for i in range(0,nodes[0])]
            plt.plot(water,yw,'r',linewidth=1)

            ii=np.where(snow[0,] != -np.inf)
            sc=1
            ii=ii[0]
            ci=ii.size
            if ci > 1:
                ys=[float(i)*dzs*sc for i in range(0,ci)]
                for j in range(0,ci-1):
                    ys[j]=ys[j]/snow[1,ii[j]]
                ys[0]=dzs/0.5
                tt=snow[0,0:ci]
                plt.axhline(y=max(ys),color='m',linewidth=0.5)
                plt.plot(tt,ys,'r',linewidth=1)

            ii=np.where(snow[0,] == -np.inf)
            ii=ii[0]
            ci=ii.size
            if ci > 0:
                slush[0,ii]=0

            ii=np.where(np.logical_and(slush[0,] != 0,slush[1,] <= 0.1))
            ii=ii[0]
            ci=ii.size
            if ci > 0:
                if ii[ci-1] >= len(ys):
                    n=ii[ci-1]-1
                else:
                    n=ii[ci-1]

            ii=np.where(np.logical_and(slush[0,] != 0,slush[1,] > 0.1))
            ii=ii[0]
            ci=ii.size
            if ci > 0:
                if ii[ci-1] >= len(ys):
                    n=ii[ci-1]-1
                else:
                    n=ii[ci-1]

            pdf.savefig()
            plt.close()

6 Water
[4. 4. 4. 4. 4.]
Prec 0.0
Snow,Water 0.0 0.0
6 Water
[3.24480394 3.99907857 3.99916228 3.99916279 3.99916279]
Prec 0.0
Snow,Water 0.0 0.0
6 Water
[2.41666554 3.99367006 3.99831237 3.99832699 3.99832702]
Prec 0.0
Snow,Water 0.0 0.0
6 Water
[1.67772226 3.98341362 3.99742288 3.99749256 3.99749281]
Prec 0.0
Snow,Water 0.0 0.0
6 Water
[0.92231834 3.96888372 3.99646701 3.99665905 3.99666003]
Prec 0.0
Snow,Water 0.0 0.0
6 Water
[0.21071354 3.95004006 3.99542009 3.99582601 3.9958287 ]
Prec 0.0
Snow,Water 0.0 0.0
6 Water
[-0.49368959  3.92718903  3.99425841  3.99499279  3.99499874]
Prec 0.0
Snow,Water 0.0 0.0
6 Water
[-1.20683178  3.9004184   3.99295977  3.99415862  3.99417015]
Prec 0.0
Snow,Water 0.0 0.0
6 Water
[-1.84632187  3.86973168  3.99150256  3.99332268  3.99334293]
Prec 0.0
Snow,Water 0.0 0.0
6 Water
[-2.12422264  3.83564579  3.98986689  3.99248388  3.99251694]
Prec 0.0
Snow,Water 0.0 0.0
6 Water
[-1.60410423  3.80039878  3.98804076  3.99164056  3.99169154]
Prec 0.0
Snow,Water 

2017-12-09 03:00:00 Ice
[-13.58237102 -15.59201633   0.                 -inf         -inf] 0.0
Prec 0.0
Snow,Water 0.0 0.0
2017-12-09 04:00:00 Ice
[-20.94084383 -13.58237102   0.                 -inf         -inf] 0.0
Prec 0.0
Snow,Water 0.0 0.0
2017-12-09 05:00:00 Ice
[-19.7891352  -20.94084383   0.                 -inf         -inf] 0.0
Prec 0.0
Snow,Water 0.0 0.0
2017-12-09 06:00:00 Ice
[-20.86440052 -19.7891352    0.                 -inf         -inf] 0.0
Prec 0.0
Snow,Water 0.0 0.0
2017-12-09 07:00:00 Ice
[-21.94689495 -20.86440052   0.                 -inf         -inf] 0.0
Prec 0.0
Snow,Water 0.0 0.0
2017-12-09 08:00:00 Ice
[-16.99943738 -21.94689495   0.                 -inf         -inf] 0.0
Prec 0.0
Snow,Water 0.0 0.0
2017-12-09 09:00:00 Ice
[-18.6440789  -16.99943738   0.                 -inf         -inf] 0.0
Prec 0.0
Snow,Water 0.0 0.0
2017-12-09 10:00:00 Ice
[-20.7546781 -18.6440789   0.               -inf        -inf] 0.0
Prec 0.0
Snow,Water 0.0 0.0
2017-12-09 11:00:00 I

Prec 0.0014
Snow,Water 0.0028189510005579145 0.0008372
III 0.001650644444444444
II 0.0024878444444444443
I 0.0024878444444444443
GGG 0.0030555555555555557
GG 0.0055434
2017-12-11 17:00:00 Snow
[5.73833102e-06 0.00000000e+00 0.00000000e+00] 0.0055434
Prec 0.001
Snow,Water 0.0008142108785100739 0.000498
III 0
II 0.000498
I 0.000498
GGG 0.0055434
GG 0.006041400000000001
2017-12-11 18:00:00 Snow
[8.52324326e-07 0.00000000e+00 0.00000000e+00] 0.006041400000000001
Prec 0.0032
Snow,Water 0.0027360675369710666 0.0012736000000000002
III 0
II 0.0012736000000000002
I 0.0012736000000000002
GGG 0.006041400000000001
GGG 0.006111111111111111
GG 0.007315
2017-12-11 19:00:00 Snow
[2.38212704e-07 0.00000000e+00 0.00000000e+00 0.00000000e+00] 0.007315
Prec 0.0042
Snow,Water 0.002972716609031005 0.0014616
III 0
II 0.0014616
I 0.0014616
GGG 0.007315
GGG 0.007315
GG 0.0087766
2017-12-11 20:00:00 Snow
[-4.88533753 -5.77924526 -6.76425378 -7.81704139 -8.89236584] 0.0087766
Prec 0.0049
Snow,Water 0.00146144916

2017-12-13 04:00:00 Snow
[-25.49328707 -25.87330395 -26.27061367 -26.68193236 -27.10386036
 -27.53291034 -27.96553607] 0.018333333333333333
Prec 0.0
Snow,Water 0.00217264916932279 0.0
III 0.0
II 0.0
I 0.0
2017-12-13 05:00:00 Snow
[-26.90933408 -27.1819373  -27.46694545 -27.76200287 -28.06467082
 -28.37244768 -28.6827896 ] 0.018333333333333333
Prec 0.0
Snow,Water 0.00217264916932279 0.0
III 0.0
II 0.0
I 0.0
2017-12-13 06:00:00 Snow
[-27.72057544 -27.87385735 -28.03411442 -28.20002209 -28.37020908
 -28.54326876 -28.71777074] 0.018333333333333333
Prec 0.0
Snow,Water 0.00217264916932279 0.0
III 0.0
II 0.0
I 0.0
2017-12-13 07:00:00 Snow
[-28.47236064 -28.61963876 -28.77361883 -28.93302817 -29.09654923
 -29.26283045 -29.43049748] 0.018333333333333333
Prec 0.0
Snow,Water 0.00217264916932279 0.0
III 0.0
II 0.0
I 0.0
2017-12-13 08:00:00 Snow
[-29.84788185 -30.12569451 -30.41496432 -30.71443367 -31.02162735
 -31.33400634 -31.64898874] 0.018333333333333333
Prec 0.0
Snow,Water 0.00217264916932279 

Prec 0.0
Snow,Water 0.0021178900011041796 0.0
III 0.0
II 0.0
I 0.0
2017-12-14 23:00:00 Snow
[-7.00326002 -6.89852432 -6.78902258 -6.67565985 -6.55937311 -6.4411235
 -6.32188838] 0.018333333333333333
Prec 0.0
Snow,Water 0.0021178900011041796 0.0
III 0.0
II 0.0
I 0.0
2017-12-15 00:00:00 Snow
[-6.63380097 -6.5629705  -6.48891687 -6.41225213 -6.33360995 -6.25364032
 -6.17300421] 0.018333333333333333
Prec 0.0
Snow,Water 0.0021178900011041796 0.0
III 0.0
II 0.0
I 0.0
2017-12-15 01:00:00 Snow
[-6.85762234 -6.90760178 -6.95985555 -7.01395177 -7.06944332 -7.12587155
 -7.18277005] 0.018333333333333333
Prec 0.0
Snow,Water 0.0021178900011041796 0.0
III 0.0
II 0.0
I 0.0
2017-12-15 02:00:00 Snow
[-6.7985967  -6.78376831 -6.76826514 -6.75221535 -6.73575157 -6.71900989
 -6.70212868] 0.018333333333333333
Prec 0.0
Snow,Water 0.0021178900011041796 0.0
III 0.0
II 0.0
I 0.0
2017-12-15 03:00:00 Snow
[-7.0085481  -7.05271299 -7.09888763 -7.14669037 -7.1957261  -7.24558953
 -7.29586853] 0.018333333333333333
P

...

Snow,Water 0.002126988933054982 0.0
III 0.0
II 0.0
I 0.0
2018-01-20 00:00:00 Snow
[ -9.55421831  -9.55839536  -9.56473681  -9.5732735   -9.58404565
  -9.59710195  -9.61249828  -9.63029632  -9.6505619   -9.67336321
  -9.69876877  -9.72684535  -9.75765571  -9.7923006   -9.83084121
  -9.87331791  -9.91974595  -9.97011144 -10.02436763 -10.08243182
 -10.14418288 -10.20945951 -10.27805943 -10.34973948 -10.42421676
 -10.50117076 -10.58024648 -10.66105856 -10.7431962  -10.82622883
 -10.90971238] 0.03361111111111111
Prec 0.0001875
Snow,Water 0.002314488933054982 0.0
III 0.0
II 0.0
I 0.0
2018-01-20 01:00:00 Snow
[ -9.71700829  -9.72722693  -9.74302014  -9.76489376  -9.79353767
  -9.82983616  -9.87487905  -9.92997236  -9.99664695 -10.07666365
 -10.17201302 -10.28490795 -10.41776741 -10.58163462 -10.7803467
 -11.01790223 -11.29832391 -11.6254913  -12.00294877 -12.43369648
 -12.91997514 -13.46305707 -14.06305764 -14.71878149 -15.42761712
 -16.18549126 -16.98689129 -17.82495966 -18.69165935 -19.5780

2018-01-20 18:00:00 Snow
[-12.13034722 -12.12047768 -12.10558212 -12.08571991 -12.06097112
 -12.03143685 -11.99723982 -11.95852475 -11.9154588  -11.86823186
 -11.81705656 -11.76216809 -11.70382359 -11.64111164 -11.57445081
 -11.50429159 -11.43111015 -11.3554004  -11.27766438 -11.19840128
 -11.11809528 -11.03720282 -10.95613973 -10.87526912 -10.79489053
 -10.71523131 -10.63644074 -10.5585876  -10.48166155 -10.40557838
 -10.33018922 -10.25529324 -10.1806532 ] 0.03361111111111111
Prec 0.0001875
Snow,Water 0.0006875566009028281 0.0
III 0.0
II 0.0
I 0.0
2018-01-20 19:00:00 Snow
[-11.87876535 -11.87178124 -11.86129681 -11.8474405  -11.83038721
 -11.81036093 -11.78763763 -11.76254849 -11.73548299 -11.7068918
 -11.67728898 -11.64725322 -11.61742762 -11.58886047 -11.5626314
 -11.53991533 -11.52196043 -11.5100581  -11.5055053  -11.50955978
 -11.52338956 -11.54801868 -11.58427186 -11.63272116 -11.69363825
 -11.76695571 -11.85224091 -11.94868511 -12.05510996 -12.16999226
 -12.29150674 -12.41758525

2018-01-21 12:00:00 Snow
[-14.47776068 -14.47856745 -14.46783522 -14.45346204 -14.43544674
 -14.41378754 -14.38848191 -14.35952621 -14.32691533 -14.29064226
 -14.25069756 -14.20706885 -14.1597403  -14.1073001  -14.04971355
 -13.98693854 -13.91892689 -13.84562686 -13.76698722 -13.68296295
 -13.59352266 -13.49865754 -13.39839159 -13.29279266 -13.18198356
 -13.06615246 -12.94556176 -12.82055448 -12.69155737 -12.55908022
 -12.42371089 -12.28610608 -12.14697812 -12.0070785 ] 0.03361111111111111
Prec 0.000125
Snow,Water 0.0007500273793354208 0.0
III 0.0
II 0.0
I 0.0
2018-01-21 13:00:00 Snow
[-14.26778794 -14.26567725 -14.25377963 -14.2377267  -14.21740116
 -14.19264888 -14.16327408 -14.12903407 -14.08963354 -14.04471879
 -13.99387222 -13.93660758 -13.87236643 -13.7976685  -13.7115062
 -13.6127423  -13.50012793 -13.37233083 -13.22797459 -13.06568882
 -12.88416954 -12.68224802 -12.45896526 -12.21364864 -11.94598612
 -11.65609304 -11.34456653 -11.01252244 -10.6616109  -10.29400722
  -9.91237699

2018-01-22 03:00:00 Snow
[-12.65523906 -12.66870798 -12.68904133 -12.71616539 -12.74997887
 -12.79035061 -12.83711687 -12.89007834 -12.94899701 -13.013593
 -13.08354172 -13.15847132 -13.23796088 -13.32306989 -13.413057
 -13.50709063 -13.60425697 -13.70357259 -13.80400171 -13.9044784
 -14.00393321 -14.10132375 -14.19566811 -14.2860799  -14.37180309
 -14.45224488 -14.52700448 -14.59589569 -14.65896157 -14.71647952
 -14.76895586 -14.81710934 -14.86184405 -14.9042126  -14.94537138] 0.03361111111111111
Prec 0.00015619999999999997
Snow,Water 0.0002498273793354218 0.0
III 0.0
II 0.0
I 0.0
2018-01-22 04:00:00 Snow
[-13.00156303 -13.0122337  -13.02837787 -13.04999434 -13.07708781
 -13.10967474 -13.14779048 -13.1914976  -13.24089531 -13.29612946
 -13.35740308 -13.42498674 -13.49922834 -13.58321593 -13.67772009
 -13.78369107 -13.90225481 -14.03469768 -14.18243846 -14.34698642
 -14.52988526 -14.73264376 -14.95665509 -15.20310798 -15.47289431
 -15.76651845 -16.08401447 -16.42487758 -16.78801587 -17

2018-01-22 17:00:00 Snow
[-16.38803758 -16.3760886  -16.35752392 -16.33255987 -16.30110256
 -16.26303745 -16.21823151 -16.16653576 -16.10778822 -16.04181718
 -15.96844482 -15.88749114 -15.79877811 -15.69916567 -15.58843859
 -15.4664113  -15.33293691 -15.18791636 -15.03130727 -14.86313244
 -14.68348769 -14.49254889 -14.29057789 -14.0779272  -13.85504315
 -13.62246747 -13.380837   -13.13088158 -12.87341999 -12.60935395
 -12.3396603  -12.06538143 -11.78761417 -11.50749741 -11.22619867] 0.03361111111111111
Prec 0.00015619999999999997
Snow,Water 0.0024209602502455103 0.0
III 0.0
II 0.0
I 0.0
2018-01-22 18:00:00 Snow
[-16.00887269 -15.99622134 -15.97716172 -15.9518264  -15.92040323
 -15.88314471 -15.84037949 -15.79252572 -15.7401058  -15.68376214
 -15.62427323 -15.56256926 -15.49974632 -15.43713598 -15.37683894
 -15.32130807 -15.27333369 -15.23600903 -15.21267328 -15.20683061
 -15.22204504 -15.26181282 -15.32941586 -15.4277622  -15.55922122
 -15.7254632  -15.92731373 -16.16463409 -16.4362379

In [77]:
# plot time series
pp = PdfPages('../data/processed/lakeice_series.pdf')

plt.xticks(rotation='vertical')
plt.ylabel('Ice thickness(cm)')
plt.plot_date(df['Date'],icethick*100,'r',linewidth=1)
pp.savefig()
plt.clf()

# snow
plt.xticks(rotation='vertical')
plt.ylim(-3,max(snowthick)*100+3)
plt.ylabel('Snow depth (cm)')
plt.plot_date(df['Date'],snowthick*100,'r',linewidth=1)
pp.savefig()
plt.clf()

# temp
plt.xticks(rotation='vertical')
plt.ylabel('Temperature (deg C)')
plt.plot_date(df['Date'],df['Tair'],'k',linewidth=1)

# ii=np.where(mtemp[0,]!=-np.inf)
# plt.ylim(min(mtemp[ii]),max(mtemp[0,]))
# plt.ylabel('Temperature (deg C)')
# ii=np.where(mtemp[0,]!=-np.inf)
# ii=ii[0]
# plt.plot(dt[ii],mtemp[0,ii],'r',linewidth=1)
# ii=np.where(mtemp[1,]!=-np.inf)
# ii=ii[0]
# plt.plot(dt[ii],mtemp[1,ii],linewidth=1)
# ii=np.where(mtemp[2,]!=-np.inf)
# ii=ii[0]
# plt.plot(dt[ii],mtemp[2,ii],'k',linewidth=1)
pp.savefig()
plt.clf()

# albedo
plt.xticks(rotation='vertical')
plt.ylim(0,1)
plt.ylabel('Albedo (-)')
plt.plot_date(df['Date'],snow_albedo,'r',linewidth=1)
pp.savefig()
plt.clf()
pp.close()

<Figure size 432x288 with 0 Axes>

In [78]:
dfo['Icethick']=icethick*100
dfo['Tair']=df['Tair']
dfo['Slush']=slush_thick*100
dfo['Load']=load[0,]
#output csv
dfo = dfo[['Date','Time', 'Icethick', 'Slush', 'Load', 'Tair']]
dfo=dfo.round(3)
dfo.to_csv('../data/processed/results.csv',sep='\t')